In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE66843"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE66843"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE66843.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE66843.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE66843.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A cell-based model unravels drivers for hepatocarcinogenesis and targets for clinical chemoprevention"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['time post infection: Day 3 post infection', 'time post infection: Day 7 post infection', 'time post infection: Day 10 post infection'], 1: ['infection: Mock infection (control)', 'infection: HCV Jc1 infection'], 2: ['cell line: Huh7.5.1']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check for gene expression data availability
# Based on the background information, this seems to be a cell line study with HCV infection
# It's likely that gene expression data is available, but it's not explicitly confirmed in the background information
# Let's set is_gene_available to True, since it's a common type of data in GEO series
is_gene_available = True

# 2.1 Data Availability
# Looking at sample characteristics:
# Key 0: Time post infection (Day 3, 7, 10) - This is a time variable, not our trait of interest
# Key 1: Infection status (Mock vs HCV) - This could be our trait of interest as it's related to liver disease
# Key 2: Cell line (constant Huh7.5.1) - This is constant across samples

# For trait: Liver cancer could be inferred from HCV infection status
trait_row = 1  # Infection status (Mock vs HCV)

# For age: No age data is available (cell line study)
age_row = None

# For gender: No gender data is available (cell line study)
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert HCV infection status to binary values:
    1 for HCV infection (risk factor for liver cancer)
    0 for Mock infection (control)
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'HCV' in value:
        return 1  # HCV infection (risk factor for liver cancer)
    elif 'Mock' in value:
        return 0  # Control
    else:
        return None

def convert_age(value):
    # Not used but defined for completeness
    return None

def convert_gender(value):
    # Not used but defined for completeness
    return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM1633236': [0.0], 'GSM1633237': [0.0], 'GSM1633238': [1.0], 'GSM1633239': [1.0], 'GSM1633240': [1.0], 'GSM1633241': [0.0], 'GSM1633242': [0.0], 'GSM1633243': [0.0], 'GSM1633244': [1.0], 'GSM1633245': [1.0], 'GSM1633246': [1.0], 'GSM1633247': [0.0], 'GSM1633248': [0.0], 'GSM1633249': [0.0], 'GSM1633250': [1.0], 'GSM1633251': [1.0], 'GSM1633252': [1.0]}
Clinical features saved to ../../output/preprocess/Liver_Cancer/clinical_data/GSE66843.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Liver_Cancer/GSE66843/GSE66843-GPL10558_series_matrix.txt.gz
Gene data shape: (46116, 17)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers start with "ILMN_" which indicates these are Illumina probe IDs
# These are not standard human gene symbols and need to be mapped to gene symbols
# Illumina probe IDs (ILMN_) are specific to Illumina microarray platforms and require mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Probe_Id', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], '

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene mapping
# From examining the gene annotation data, we can see:
# - 'ID' column contains the Illumina probe IDs (ILMN_*)
# - 'Symbol' column contains the gene symbols we want to map to

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',  # Column containing probe IDs
    gene_col='Symbol'  # Column containing gene symbols
)

print("Gene mapping preview:")
print(gene_mapping.head())
print(f"Gene mapping shape: {gene_mapping.shape}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(
    expression_df=gene_data,  # Probe-level expression data
    mapping_df=gene_mapping   # Mapping between probes and genes
)

print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nAfter normalization:")
print(f"Shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene mapping shape: (44837, 2)
Gene expression data after mapping:
Shape: (21125, 17)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')



After normalization:
Shape: (19956, 17)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Liver_Cancer/gene_data/GSE66843.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, extract and save the clinical data since it's missing
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the background info and clinical data again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define the conversion functions from Step 2
def convert_trait(value):
    """Convert PCOS trait to binary (0 = control, 1 = PCOS)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'PCOS' in value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)
    Note: In this context, we're dealing with biological sex rather than gender identity
    Female-to-male transsexuals are biologically female (0)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower() and 'female to male' not in value.lower():
        return 1
    else:
        return 0  # Female to male transsexuals are recorded as female (0) biologically

# Extract clinical features with the correct row indices from previous steps
trait_row = 1  # Contains "disease state: PCOS"
gender_row = 0  # Contains gender information
age_row = None  # Age information is not available in this dataset

# Process and save clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Normalize gene symbols using synonym information from NCBI
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene identifiers:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains gene expression data from ovary biopsies of women with PCOS and female-to-male transsexual individuals, focusing on LH-induced gene expression."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data saved to: ../../output/preprocess/Liver_Cancer/clinical_data/GSE66843.csv
Clinical data preview:
{'GSM1633236': [0.0, 0.0], 'GSM1633237': [0.0, 0.0], 'GSM1633238': [0.0, 0.0], 'GSM1633239': [0.0, 0.0], 'GSM1633240': [0.0, 0.0], 'GSM1633241': [0.0, 0.0], 'GSM1633242': [0.0, 0.0], 'GSM1633243': [0.0, 0.0], 'GSM1633244': [0.0, 0.0], 'GSM1633245': [0.0, 0.0], 'GSM1633246': [0.0, 0.0], 'GSM1633247': [0.0, 0.0], 'GSM1633248': [0.0, 0.0], 'GSM1633249': [0.0, 0.0], 'GSM1633250': [0.0, 0.0], 'GSM1633251': [0.0, 0.0], 'GSM1633252': [0.0, 0.0]}

Normalizing gene symbols...
Gene data shape after normalization: (19956, 17)
First 10 normalized gene identifiers:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Liver_Cancer/gene_data/GSE66843.csv

Linking clinical and genetic data...
Linked data shape: (17, 19958)
Linked data preview (first 5 rows, 5 columns):
            Liver_Cancer  Gender        A1BG   A1BG-AS1         A1CF
GSM1633236           0.0     0.0  175.902881  84.928332  2058.146767
GSM1633237           0.0     0.0  175.467437  87.225868  1601.515698
GSM1633238           0.0     0.0  185.163824  89.011397  1713.037609
GSM1633239           0.0     0.0  175.411354  85.778853  1508.079221
GSM1633240           0.0     0.0  181.091704  84.390770  1556.390170

Handling missing values...
Samples with missing trait values: 0 out of 17
Genes with ≤20% missing values: 19956 out of 19956
Samples with ≤5% missing gene values: 17 out of 17


Linked data shape after handling missing values: (17, 19958)

Checking for bias in dataset features...
Quartiles for 'Liver_Cancer':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Liver_Cancer' in this dataset is severely biased.

For the feature 'Gender', the least common label is '0.0' with 17 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Dataset deemed not usable for associative studies. Linked data not saved.
